In [1]:
import tensorflow as tf
import numpy as np
from datetime import datetime   # date stamp the log directory
import json  # for saving and loading hyperparameters
import os, sys, re
import time

import absl
import absl.logging as logging
from tf2_models.matrix_caps import *
from util.config_util import get_model_params, get_task_params, get_train_params
from tf2_models.trainer import Trainer
from util.models import MODELS 
from util.tasks import TASKS
from notebook_utils import *

import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd

import seaborn as sns; sns.set()

from tqdm import tqdm

gfile = tf.io.gfile
flags = absl.app.flags

[nltk_data] Downloading package punkt to /home/dehghani/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [8]:
class CapsConfig(object):
  def __init__(self,
               **kwargs):
    self.output_dim = 10
    self.A = 32
    self.B = 32
    self.C = 16
    self.D = 16
    self.epsilon = 1e-9
    self.l2 = 0.0000002
    self.final_lambda = 0.01
    self.iter_routing = 2

In [9]:

def spread_loss(scores, y, global_step):
    """Spread loss.

    "In order to make the training less sensitive to the initialization and 
    hyper-parameters of the model, we use “spread loss” to directly maximize the 
    gap between the activation of the target class (a_t) and the activation of the 
    other classes. If the activation of a wrong class, a_i, is closer than the 
    margin, m, to at then it is penalized by the squared distance to the margin."

    See Hinton et al. "Matrix Capsules with EM Routing" equation (3).

    Author:
    Ashley Gritzman 19/10/2018  
    Credit:
    Adapted from Suofei Zhang's implementation on GitHub, "Matrix-Capsules-
    EM-Tensorflow"
    https://github.com/www0wwwjs1/Matrix-Capsules-EM-Tensorflow  
    Args: 
    scores: 
      scores for each class 
      (batch_size, num_class)
    y: 
      index of true class 
      (batch_size, 1)  
    Returns:
    loss: 
      mean loss for entire batch
      (scalar)
    """
  
    batch_size = tf.shape(scores)[0]

    # margin = 0.2 + .79 * tf.sigmoid(tf.minimum(10.0, step / 50000.0 - 4))
    # where step is the training step. We trained with batch size of 64."
    m_min = 0.2
    m_delta = 0.79
    m = (m_min 
         + m_delta * tf.sigmoid(tf.minimum(10.0, global_step / 50000.0 - 4)))

    num_class = tf.shape(scores)[-1]

    y = tf.one_hot(y, num_class, dtype=tf.float32)

    # Get the score of the target class
    # (64, 1, 5)
    scores = tf.reshape(scores, shape=[batch_size, 1, num_class])
    # (64, 5, 1)
    y = tf.expand_dims(y, axis=2)
    # (64, 1, 5)*(64, 5, 1) = (64, 1, 1)
    at = tf.matmul(scores, y)

    # Compute spread loss, paper eq (3)
    loss = tf.math.square(tf.maximum(0., m - (at - scores)))

    # Sum losses for all classes
    # (64, 1, 5)*(64, 5, 1) = (64, 1, 1)
    # e.g loss*[1 0 1 1 1]
    loss = tf.matmul(loss, 1. - y)

    # Compute mean
    loss = tf.reduce_mean(loss)

    return loss


In [13]:
chkpt_dir='../tf_ckpts'
task_name='mnist'
strategy = tf.distribute.MirroredStrategy(devices=['/gpu:0','/gpu:1','/gpu:2','/gpu:3'])

with strategy.scope(): 
    task = TASKS[task_name](get_task_params(batch_size=128), data_dir='../data')
    
    
for x,y in task.train_dataset:
    print(x.shape, y.shape)
    break

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')


(128, 28, 28, 1) (128,)


In [14]:
with strategy.scope(): 
    loss = task.get_loss_fn()
    optimizer = optimizer=tf.keras.optimizers.Adam(learning_rate=0.001)


In [15]:

#example_x, example_y = next(iter(task.train_dataset))
#outputs = model(example_x, training=True)
    
with strategy.scope():
    model = MatrixCaps(CapsConfig())
    out = model(x) #.build(input_shape=task.input_shape())
    model.compile(optimizer=optimizer,
                  loss=loss,
                  metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

model.summary()

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


Model: "matrix_caps"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_1 (Batch multiple                  4         
_________________________________________________________________
conv2d_3 (Conv2D)            multiple                  832       
_________________________________________________________________
conv2d_4 (Conv2D)            multiple                  16896     
_________________________________________________________________
conv2d_5 (Conv2D)            multiple                  1056      
_________________________________________________________________
conv_caps1 (ConvCaps)        multiple                  73760     
_________________________________________________________________
conv_caps2 (ConvCaps)        multiple                  36896     
_________________________________________________________________
class_caps (FcCaps)          multiple                  

In [ ]:
history = model.fit(task.train_dataset,
                    epochs=20,
                    steps_per_epoch=task.n_train_batches)

print('\nhistory dict:', history.history)

Train for 468 steps
Epoch 1/20
INFO:tensorflow:batch_all_reduce: 17 all-reduces with algorithm = nccl, num_packs = 1, agg_small_grads_max_bytes = 0 and agg_small_grads_max_group = 10


INFO:tensorflow:batch_all_reduce: 17 all-reduces with algorithm = nccl, num_packs = 1, agg_small_grads_max_bytes = 0 and agg_small_grads_max_group = 10


468/468 [==============================] - 2163s 5s/step - loss: 1.8869 - sparse_categorical_accuracy: 0.9343
Epoch 2/20
468/468 [==============================] - 2138s 5s/step - loss: 1.8618 - sparse_categorical_accuracy: 0.9475
Epoch 3/20
468/468 [==============================] - 2143s 5s/step - loss: 1.8366 - sparse_categorical_accuracy: 0.9538
Epoch 4/20
399/468 [========================>.....] - ETA: 5:16 - loss: 1.8134 - sparse_categorical_accuracy: 0.9550

In [ ]:
#with strategy.scope():
history = model.evaluate(task.test_dataset,
                    steps=task.n_test_batches)

print('\nhistory dict:', history.history)